<a href="https://colab.research.google.com/github/ErrorOfTheGeneration/pyBotTG_google/blob/main/pyBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
import random   # для рандомных ответов из словаря ответов
import nltk     # для работы с опечатками
import json


In [50]:
# далее обозначено зачем 1-я библиотека. 2-ю погуглить
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

In [53]:
with open('/content/BIG_BOT_CONFIG.json', 'r') as f:  # json для хранения словарей (поискать справку)
  BOT_CONFIG = json.load(f)
# словарь словарей. Нужен нам для того, чтобы определять, что хочет пользователь
# и соттветствующе отвечать ему
# BOT_CONFIG = {
#     'intents': {
#         'hello': {
#             'examples': ['Привет!', 'Хай', 'Добрый день', 'здравствуй', 'здравствуйте',
#                          'приветик', 'здрасте', 'здорова'], # примеры приветствия
#             'resposes': ['Доброго времени суток', 'Привет', 'приветствую', 'Здравствуй']  #ответы бота
#         },
#         'bye': {
#             'examples': ['Пока', 'До свидания', 'Увидимся', 'Счастливо', 'до новых встреч'],
#             'resposes': ['Счастливо', 'Хаюхай', 'Удачного дня', 'Приходите ещё', 'Будьте здоровы', 'До новых встреч!']
#         }
#     }
# }

In [ ]:
len(BOT_CONFIG['intents'])

In [54]:


def clean(text): #принимает текст, возвращает чищенный текст ниж.регистр и без знаков препинания
  return ''.join([simbol for simbol in text.lower() if simbol in 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя '])

# функция match нужна для того, чтобы бот не был чувствителен к регистру или знакам препинания

# nltk позволяет вычислять расстояние Ливенштейна 
# между строками, т.е. кол-во редакторских исправлений, 
# чтобы из одного слова получить другое
def match(example, text):      
  return nltk.edit_distance(clean(text),clean(example)) / len(example) < 0.4

# пробегаем по всему нашему словарю и ищем, в какой интент (намерение) попадает
# введённое пользователем слово
def get_intent(text):
  for intent, value in BOT_CONFIG['intents'].items():
    for example in value['examples']:
      if match(example, text):
        return random.choice(value['resposes'])
  return'Я не понимаю, чего от меня хотят'

In [ ]:
question = input()            # просим пользователя ввести
answer = get_intent(question)
print(answer)

In [19]:
while question != 'выход':
  question = input()            
  answer = get_intent(question)
  print(answer)

возьмем библиотеку scikit-learn. В ней есть векторайзеры - специальные объекты,
которые превращают тексты в вектора

In [ ]:
vectorizer = CountVectorizer()
vectorizer

In [21]:
# Сделаем дата-сет (набор данных для машинного обучения) из файла BOT_CONFIG
# нам нужны пары объект-ответ
# икс - для объектов обучающей выборки. там - вектора всех текстов 
# игрек - ответы для всех векторов, записанные в том же самом порядке
# X = [v1, v2, v3]
# Y = [int1, int2, int3]

In [58]:
X = []
Y = []

for intent, value in BOT_CONFIG['intents'].items():
  examples = set(value['examples']) # *1
  X = X + value['examples']
  Y = Y + [intent] * len(value['examples'])
# *1 преобразуем список в множество на случай неуникальных экземплс, чтобы не 
# не затруднять обучение машине. Теперь элементы не повторяются

In [59]:
# Векторизуем дата-сет. Эти вектора можно подавать на вход модели
X_transformed = vectorizer.fit_transform(X)

In [60]:
# создадим объект форест классифаер. Фит - обучить
classifier = RandomForestClassifier()

In [61]:
# обучим объект на X_transformed и Y. То есть ему надо дать вектора и ответ на них.
# Он попробует найти зависимость, чтобы для новых векторов уж езнать ответ
classifier.fit(X_transformed, Y)

RandomForestClassifier()

In [62]:
# теперь мы можем делать предсказания (predict)
text = 'пока'
classifier.predict(vectorizer.transform([text]))

array(['bye'], dtype='<U5')

In [31]:
# Чтобы что-то векторизовать, надо понимать, о каком словаре идет речь.
# Значит, сначала векторайзер нужно обучить. Для обучения ему нужно несколько текстов
vectorizer.fit(['Доброго времени суток', 'Привет', 'приветствую', 'Здравствуй'])

CountVectorizer()

Заметим что векторайзер не зависит от регистра. Любому тексту он может поставить в соответствие вектор. И всем текстам надо поставить вектора, чтобы их показать модели машинного обучения, а она на основе векторов попытается понять, как зависит класс текста, намерение пользователя, от того, на каких местах какие числа в этих векторах располагаются.


In [32]:
vectorizer.transform(['Доброго времени суток', 'Привет', 'приветствую', 'Здравствуй'])[0].toarray()

array([[1, 1, 0, 0, 0, 1]])